## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [2]:
data = list(range(1, 1001))

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [4]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [5]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [6]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [7]:
# count
rdd.count()

1000

In [8]:
# first
rdd.first()

1

In [9]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [11]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [12]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [13]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([5.37023808e-01, 9.60883371e-01, 1.76978542e+00, 2.29973224e+00,
       2.72302063e+00, 4.26628458e+00, 6.09530996e+00, 7.26304602e-01,
       1.29118526e+00, 9.29684326e+00, 4.90613421e-01, 3.16758817e+00,
       2.78963812e+00, 6.30613053e+00, 1.02015585e+01, 9.87793042e+00,
       1.55868092e+01, 2.39895483e+00, 2.98005427e+00, 2.89208455e+00,
       1.58227418e+01, 1.09509111e+01, 1.70192629e+01, 1.10269456e+01,
       2.29407903e+00, 1.47619966e+01, 2.08347759e+01, 1.22069868e+01,
       1.96619457e+01, 1.54821998e+01, 5.09876234e+00, 5.32776184e-01,
       1.71014447e+01, 1.60588403e+01, 2.56384689e+01, 4.78748410e+00,
       6.43300946e+00, 3.79164499e-01, 3.68280302e+01, 1.40413781e+01,
       3.46665465e+01, 2.41314707e+01, 8.53389066e+00, 2.65377761e+01,
       3.48820942e+00, 1.73036481e+01, 4.65970083e+01, 2.31363501e+01,
       5.58275932e+00, 4.47588898e+01, 4.65922306e+01, 3.43487628e+01,
       2.63306887e+01, 1.84610273e+01, 4.13955020e+01, 4.30273326e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [15]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(5)

[0.5370238082864995,
 0.9608833711018276,
 1.7697854206321484,
 2.2997322417508435,
 2.7230206280214238]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [17]:
def sales_tax(num):
    return num/1.08

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [18]:
# perform action to retrieve rdd values
revenue_minus_tax.take(5)

[0.49724426693194396,
 0.8897068250942848,
 1.6386902042890261,
 2.129381705324855,
 2.5213153963161328]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [19]:
discounted = revenue_minus_tax.map(lambda x: x/1.1)

In [20]:
discounted.take(10)

[0.45204024266540355,
 0.8088243864493497,
 1.4897183675354781,
 1.9358015502953225,
 2.2921049057419385,
 3.591148636205577,
 5.130732292852808,
 0.6113675102132703,
 1.0868562802783432,
 7.825625641414946]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [21]:
price_items.map(sales_tax).map(lambda x: x/1.1).top(15)

[799.9522082876234,
 792.1032445757952,
 784.3805157389494,
 741.968093017081,
 738.759936395931,
 738.1517742314119,
 736.7874057663447,
 735.4907086552062,
 713.2517839869398,
 712.1643027505478,
 711.2343447055401,
 706.9792841889185,
 698.164466970357,
 687.3297224871085,
 682.109316665911]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [22]:
discounted.toDebugString()

b'(10) PythonRDD[11] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [24]:
mapped = price_items.map(lambda x: (x, x/1.08/1.1))
print(mapped.count())
print(mapped.take(10))

1000
[(0.5370238082864995, 0.45204024266540355), (0.9608833711018276, 0.8088243864493497), (1.7697854206321484, 1.4897183675354781), (2.2997322417508435, 1.9358015502953225), (2.7230206280214238, 2.2921049057419385), (4.266284579812226, 3.591148636205577), (6.095309963909136, 5.130732292852808), (0.7263046021333652, 0.6113675102132703), (1.2911852609706718, 1.0868562802783432), (9.296843262000957, 7.825625641414946)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [25]:
flat_mapped = price_items.flatMap(lambda x: (x, x/1.08/1.1))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.5370238082864995, 0.45204024266540355, 0.9608833711018276, 0.8088243864493497, 1.7697854206321484, 1.4897183675354781, 2.2997322417508435, 1.9358015502953225, 2.7230206280214238, 2.2921049057419385]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [26]:
# use the filter function
selected_items = price_items.filter(lambda x: x/1.08/1.1 > 300)

# calculate total remaining in inventory 
selected_items.count()

270

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [27]:
selected_items.reduce(lambda x, y: x + y)

144837.40646910033

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [28]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(6, 379.4913368199466),
 (6, 394.3027755333483),
 (33, 393.0883510459122),
 (9, 408.5588827659725),
 (34, 416.41646744234174),
 (34, 422.9655233718477),
 (23, 393.2256273717435)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [31]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x + y)

In [33]:
# sort the users from highest to lowest spenders
sorted_total_spent = total_spent.sortBy(lambda x: x[1], ascending=False)
sorted_total_spent.take(10)

[(28, 7500.593680948749),
 (50, 6178.309331470754),
 (23, 6094.029267708662),
 (34, 6061.565927439785),
 (43, 5888.1013706719805),
 (21, 5869.121074264755),
 (8, 5732.856633698081),
 (46, 5584.726559462275),
 (2, 5486.613535323924),
 (49, 5129.710448069038)]

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [37]:
n_items = sales_data.countByKey()
sorted(n_items.items(), key=lambda kv:kv[1], reverse=True)

[(34, 13),
 (23, 12),
 (8, 12),
 (21, 11),
 (28, 11),
 (38, 10),
 (32, 10),
 (20, 10),
 (50, 10),
 (49, 10),
 (2, 10),
 (43, 9),
 (22, 8),
 (11, 8),
 (36, 8),
 (46, 8),
 (25, 6),
 (1, 6),
 (39, 6),
 (4, 6),
 (6, 5),
 (15, 5),
 (30, 5),
 (26, 5),
 (24, 5),
 (33, 4),
 (9, 4),
 (37, 4),
 (3, 4),
 (40, 4),
 (29, 4),
 (35, 4),
 (31, 4),
 (12, 4),
 (42, 4),
 (13, 3),
 (18, 3),
 (10, 3),
 (19, 3),
 (47, 2),
 (14, 2),
 (7, 2),
 (41, 1),
 (44, 1),
 (27, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
